<a href="https://colab.research.google.com/github/lemmingy/My_pipelines/blob/main/liner_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install category_encoders

In [2]:
%cd /content/drive/My Drive/Colab Notebooks/Kaggle/Library/Input_data/liner_model

/content/drive/My Drive/Colab Notebooks/Kaggle/Library/Input_data/liner_model


In [3]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)#表示列を増加。
pd.set_option('display.max_rows', 500)#表示列を増加。

df_train = pd.read_csv("train.csv")
# df_train= pd.read_csv('XXX.csv',parse_dates=['issue_d','earliest_cr_line'])#日付がある場合にはparseする。
df_train.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (9,13) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Respondent,ConvertedSalary,Hobby,OpenSource,Country,Student,Employment,FormalEducation,UndergradMajor,CompanySize,DevType,YearsCoding,YearsCodingProf,JobSatisfaction,CareerSatisfaction,HopeFiveYears,JobSearchStatus,LastNewJob,AssessJob1,AssessJob2,AssessJob3,AssessJob4,AssessJob5,AssessJob6,AssessJob7,AssessJob8,AssessJob9,AssessJob10,AssessBenefits1,AssessBenefits2,AssessBenefits3,AssessBenefits4,AssessBenefits5,AssessBenefits6,AssessBenefits7,AssessBenefits8,AssessBenefits9,AssessBenefits10,AssessBenefits11,JobContactPriorities1,JobContactPriorities2,JobContactPriorities3,JobContactPriorities4,JobContactPriorities5,JobEmailPriorities1,JobEmailPriorities2,JobEmailPriorities3,JobEmailPriorities4,JobEmailPriorities5,JobEmailPriorities6,JobEmailPriorities7,UpdateCV,Currency,SalaryType,CurrencySymbol,CommunicationTools,TimeFullyProductive,TimeAfterBootcamp,AgreeDisagree1,AgreeDisagree2,AgreeDisagree3,FrameworkWorkedWith,OperatingSystem,NumberMonitors,CheckInCode,AdBlocker,AdBlockerDisable,AdsAgreeDisagree1,AdsAgreeDisagree2,AdsAgreeDisagree3,AdsActions,AdsPriorities1,AdsPriorities2,AdsPriorities3,AdsPriorities4,AdsPriorities5,AdsPriorities6,AdsPriorities7,AIDangerous,AIInteresting,AIResponsible,AIFuture,EthicsChoice,EthicsReport,EthicsResponsible,EthicalImplications,StackOverflowRecommend,StackOverflowVisit,StackOverflowHasAccount,StackOverflowParticipate,StackOverflowJobs,StackOverflowDevStory,StackOverflowJobsRecommend,StackOverflowConsiderMember,HypotheticalTools1,HypotheticalTools2,HypotheticalTools3,HypotheticalTools4,HypotheticalTools5,WakeTime,HoursComputer,HoursOutside,SkipMeals,ErgonomicDevices,Exercise,Gender,SexualOrientation,EducationParents,RaceEthnicity,Age,Dependents,MilitaryUS,SurveyTooLong,SurveyEasy
0,3,70841.0,Yes,Yes,United Kingdom,No,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","A natural science (ex. biology, chemistry, phy...","10,000 or more employees",Database administrator;DevOps specialist;Full-...,30 or more years,18-20 years,Moderately dissatisfied,Neither satisfied nor dissatisfied,Working in a different or more specialized tec...,I am actively looking for a job,More than 4 years ago,1.0,7.0,10.0,8.0,2.0,5.0,4.0,3.0,6.0,9.0,1.0,5.0,3.0,7.0,10.0,4.0,11.0,9.0,6.0,2.0,8.0,3.0,1.0,5.0,2.0,4.0,1.0,3.0,4.0,5.0,2.0,6.0,7.0,I saw an employer’s advertisement,British pounds sterling (£),Yearly,GBP,Confluence;Office / productivity suite (Micros...,One to three months,NaN,Agree,Agree,Neither Agree nor Disagree,Django,Linux-based,2,A few times per week,Yes,Yes,Somewhat agree,Neither agree nor disagree,Neither agree nor disagree,NaN,3.0,5.0,1.0,4.0,6.0,7.0,2.0,Increasing automation of jobs,Increasing automation of jobs,The developers or the people creating the AI,I'm excited about the possibilities more than ...,Depends on what it is,Depends on what it is,Upper management at the company/organization,Yes,10 (Very Likely),A few times per month or weekly,Yes,A few times per month or weekly,Yes,"No, I have one but it's out of date",7,Yes,A little bit interested,A little bit interested,A little bit interested,A little bit interested,A little bit interested,Between 6:01 - 7:00 AM,5 - 8 hours,30 - 59 minutes,Never,Ergonomic keyboard or mouse,Daily or almost every day,Male,Straight or heterosexual,"Bachelor’s degree (BA, BS, B.Eng., etc.)",White or of European descent,35 - 44 years old,Yes,NaN,The survey was an appropriate length,Somewhat easy
1,7,21426.0,Yes,No,South Africa,"Yes, part-time",Employed full-time,Some college/university study without earning ...,"Computer science, computer engineering, or sof...","10,000 or more employees",Data or business analyst;Desktop or enterprise...,6-8 years,0-2 years,Slightly satisfied,Moderately satisfied,Working in a different or more specialized tec...,"I’m not actively looking, but I am open to new...",Between 1 and 2 years ago,8.0,5.0,7.0,1.0,2.0,6.0,4.0,3.0,10.0,9.0,1.0,10.0,2.0,4.0,8.0,3.0,11.0,7.0,5.0,9.0,6.0,2.0,1.0,4.0,5.0,3.0,7.0,3.0,6.0,2.0,1.0,4.0,5.0,M

In [4]:
# Testのデータの傾向をよくみる。Trainingデータと比べて違う所はないか？トレーニングデータをそのまま使って良いのか？正しいパーティションは何か？
df_test = pd.read_csv("test.csv")
df_test.head()

,Respondent,Hobby,OpenSource,Country,Student,Employment,FormalEducation,UndergradMajor,CompanySize,DevType,YearsCoding,YearsCodingProf,JobSatisfaction,CareerSatisfaction,HopeFiveYears,JobSearchStatus,LastNewJob,AssessJob1,AssessJob2,AssessJob3,AssessJob4,AssessJob5,AssessJob6,AssessJob7,AssessJob8,AssessJob9,AssessJob10,AssessBenefits1,AssessBenefits2,AssessBenefits3,AssessBenefits4,AssessBenefits5,AssessBenefits6,AssessBenefits7,AssessBenefits8,AssessBenefits9,AssessBenefits10,AssessBenefits11,JobContactPriorities1,JobContactPriorities2,JobContactPriorities3,JobContactPriorities4,JobContactPriorities5,JobEmailPriorities1,JobEmailPriorities2,JobEmailPriorities3,JobEmailPriorities4,JobEmailPriorities5,JobEmailPriorities6,JobEmailPriorities7,UpdateCV,Currency,SalaryType,CurrencySymbol,CommunicationTools,TimeFullyProductive,TimeAfterBootcamp,AgreeDisagree1,AgreeDisagree2,AgreeDisagree3,FrameworkWorkedWith,OperatingSystem,NumberMonitors,CheckInCode,AdBlocker,AdBlockerDisable,AdsAgreeDisagree1,AdsAgreeDisagree2,AdsAgreeDisagree3,AdsActions,AdsPriorities1,AdsPriorities2,AdsPriorities3,AdsPriorities4,AdsPriorities5,AdsPriorities6,AdsPriorities7,AIDangerous,AIInteresting,AIResponsible,AIFuture,EthicsChoice,EthicsReport,EthicsResponsible,EthicalImplications,StackOverflowRecommend,StackOverflowVisit,StackOverflowHasAccount,StackOverflowParticipate,StackOverflowJobs,StackOverflowDevStory,StackOverflowJobsRecommend,StackOverflowConsiderMember,HypotheticalTools1,HypotheticalTools2,HypotheticalTools3,HypotheticalTools4,HypotheticalTools5,WakeTime,HoursComputer,HoursOutside,SkipMeals,ErgonomicDevices,Exercise,Gender,SexualOrientation,EducationParents,RaceEthnicity,Age,Dependents,MilitaryUS,SurveyTooLong,SurveyEasy
0,81,No,Yes,Germany,No,Employed full-time,Some college/university study without earning ...,"Computer science, computer engineering, or sof...",100 to 499 employees,Back-end developer;Desktop or enterprise appli...,9-11 years,9-11 years,Moderately satisfied,Extremely satisfied,Working as a product manager or project manager,"I’m not actively looking, but I am open to new...",Less than a year ago,9.0,5.0,10.0,2.0,3.0,4.0,6.0,1.0,8.0,7.0,1.0,10.0,11.0,6.0,8.0,9.0,2.0,5.0,4.0,3.0,7.0,5.0,1.0,4.0,2.0,3.0,5.0,6.0,4.0,2.0,1.0,7.0,3.0,My job status or other personal status changed,Euros (€),Monthly,EUR,Confluence;Jira;Office / productivity suite (M...,Less than a month,NaN,Agree,Strongly agree,Disagree,Spring,Windows,3,Multiple times per day,Yes,Yes,Neither agree nor disagree,Somewhat agree,Somewhat agree,Stopped going to a website because of their ad...,7.0,4.0,1.0,3.0,2.0,6.0,5.0,NaN,NaN,Nobody,"I don't care about it, or I haven't thought ab...",Depends on what it is,"Yes, but only within the company",Upper management at the company/organization,Unsure / I don't know,4,A few times per week,Yes,Less than once per month or monthly,"No, I knew that Stack Overflow had a jobs boar...","No, I know what it is but I don't have one",NaN,Yes,A little bit interested,Somewhat interested,Not at all interested,Not at all interested,Not at all interested,Between 6:01 - 7:00 AM,5 - 8 hours,1 - 2 hours,Never,NaN,I don't typically exercise,Male,Straight or heterosexual,"Secondary school (e.g. American high school, G...",White or of European descent,35 - 44 years old,Yes,NaN,The survey was too long,Neither easy nor difficult
1,85,Yes,Yes,France,"Yes, part-time",Employed part-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",10 to 19 employees,Back-end developer;Front-end developer;Student,9-11 years,0-2 years,Neither satisfied nor dissatisfied,Moderately dissatisfied,Working as a founder or co-founder of my own c...,"I’m not actively looking, but I am open to new...",Less than a year ago,1.0,10.0,4.0,5.0,2.0,6.0,3.0,7.0,8.0,9.0,1.0,3.0,9.0,11.0,10.0,8.0,4.0,7.0,6.0,2.0,5.0,5.0,1.0,4.0,2.0,3.0,5.0,6.0,1.0,2.0,4.0,7.0,3.0,I had a negative experience or interaction at ...,Euros (€),Monthly,EUR,"Slack;Other ch

In [5]:
# ターゲットを分けておく。IDも分けておく
target = "ConvertedSalary"
id = "Respondent"
try:
  df_train_y = df_train[target]
  df_train.drop(columns = target, inplace = True)
  df_train_id = df_train[id]
  df_test_id = df_test[id]
  df_train.drop(columns = id, inplace = True)
  df_test.drop(columns = id, inplace = True)  
except:
  pass

print(df_train.shape, df_test.shape, df_train_y.shape)

(42893, 112) (4809, 112) (42893,)


In [6]:
# 時間特徴量の簡易処理（あれば。事前に時間特徴量にparse_datesで変換しておく。）
# 時間を分解
def sep_time(df, col, col2=0):
  df[col+"_y"]=df[col].dt.year
  df[col+"_y"]=df[col].dt.month
  df[col+"_y"]=df[col].dt.day
  if col2!=0:
    df[col2+"_y"]=df[col2].dt.year
    df[col2+"_y"]=df[col2].dt.month
    df[col2+"_y"]=df[col2].dt.day
    df[col+"-"+col2]=df[col] - df[col2]
  return df

def time_encode(df, col):
    # この方法だと場合によって最大値が変化するデータでは正確な値は出ない
    # 例：月の日数が30日や31日の場合がある
    df[col + '_cos'] = np.cos(2 * np.pi * df[col] / df[col].max())
    df[col + '_sin'] = np.sin(2 * np.pi * df[col] / df[col].max())
    return df


In [7]:
# 数値カラム、カテゴリカラム、テキストコラムのリストを取得
def get_columns(df):
  lis_cols_num = [ col for col in df.columns if df[col].dtype == "float64" or df[col].dtype == "int64" ]
  lis_cols_cat = [ col for col in df.columns if df[col].dtype == "object" or df[col].dtype == 'O']
  return lis_cols_num, lis_cols_cat

lis_cols_num, lis_cols_cat  =  get_columns(df_train)


In [8]:
# 数値カラムの外れ値の影響を減らすためのクリッピング
def clip(df, lis_cols_num):
  for col in lis_cols_num:
    upperbound, lowerbound= np.percentile(df[col],[1,99])
    df[col]=np.clip(df[col],upperbound,lowerbound)
    return df

df_train = clip(df_train, lis_cols_num)
df_test = clip(df_test, lis_cols_num)
df_train.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:7212: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result[mask] = np.nan


,Hobby,OpenSource,Country,Student,Employment,FormalEducation,UndergradMajor,CompanySize,DevType,YearsCoding,YearsCodingProf,JobSatisfaction,CareerSatisfaction,HopeFiveYears,JobSearchStatus,LastNewJob,AssessJob1,AssessJob2,AssessJob3,AssessJob4,AssessJob5,AssessJob6,AssessJob7,AssessJob8,AssessJob9,AssessJob10,AssessBenefits1,AssessBenefits2,AssessBenefits3,AssessBenefits4,AssessBenefits5,AssessBenefits6,AssessBenefits7,AssessBenefits8,AssessBenefits9,AssessBenefits10,AssessBenefits11,JobContactPriorities1,JobContactPriorities2,JobContactPriorities3,JobContactPriorities4,JobContactPriorities5,JobEmailPriorities1,JobEmailPriorities2,JobEmailPriorities3,JobEmailPriorities4,JobEmailPriorities5,JobEmailPriorities6,JobEmailPriorities7,UpdateCV,Currency,SalaryType,CurrencySymbol,CommunicationTools,TimeFullyProductive,TimeAfterBootcamp,AgreeDisagree1,AgreeDisagree2,AgreeDisagree3,FrameworkWorkedWith,OperatingSystem,NumberMonitors,CheckInCode,AdBlocker,AdBlockerDisable,AdsAgreeDisagree1,AdsAgreeDisagree2,AdsAgreeDisagree3,AdsActions,AdsPriorities1,AdsPriorities2,AdsPriorities3,AdsPriorities4,AdsPriorities5,AdsPriorities6,AdsPriorities7,AIDangerous,AIInteresting,AIResponsible,AIFuture,EthicsChoice,EthicsReport,EthicsResponsible,EthicalImplications,StackOverflowRecommend,StackOverflowVisit,StackOverflowHasAccount,StackOverflowParticipate,StackOverflowJobs,StackOverflowDevStory,StackOverflowJobsRecommend,StackOverflowConsiderMember,HypotheticalTools1,HypotheticalTools2,HypotheticalTools3,HypotheticalTools4,HypotheticalTools5,WakeTime,HoursComputer,HoursOutside,SkipMeals,ErgonomicDevices,Exercise,Gender,SexualOrientation,EducationParents,RaceEthnicity,Age,Dependents,MilitaryUS,SurveyTooLong,SurveyEasy
0,Yes,Yes,United Kingdom,No,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","A natural science (ex. biology, chemistry, phy...","10,000 or more employees",Database administrator;DevOps specialist;Full-...,30 or more years,18-20 years,Moderately dissatisfied,Neither satisfied nor dissatisfied,Working in a different or more specialized tec...,I am actively looking for a job,More than 4 years ago,1.0,7.0,10.0,8.0,2.0,5.0,4.0,3.0,6.0,9.0,1.0,5.0,3.0,7.0,10.0,4.0,11.0,9.0,6.0,2.0,8.0,3.0,1.0,5.0,2.0,4.0,1.0,3.0,4.0,5.0,2.0,6.0,7.0,I saw an employer’s advertisement,British pounds sterling (£),Yearly,GBP,Confluence;Office / productivity suite (Micros...,One to three months,NaN,Agree,Agree,Neither Agree nor Disagree,Django,Linux-based,2,A few times per week,Yes,Yes,Somewhat agree,Neither agree nor disagree,Neither agree nor disagree,NaN,3.0,5.0,1.0,4.0,6.0,7.0,2.0,Increasing automation of jobs,Increasing automation of jobs,The developers or the people creating the AI,I'm excited about the possibilities more than ...,Depends on what it is,Depends on what it is,Upper management at the company/organization,Yes,10 (Very Likely),A few times per month or weekly,Yes,A few times per month or weekly,Yes,"No, I have one but it's out of date",7,Yes,A little bit interested,A little bit interested,A little bit interested,A little bit interested,A little bit interested,Between 6:01 - 7:00 AM,5 - 8 hours,30 - 59 minutes,Never,Ergonomic keyboard or mouse,Daily or almost every day,Male,Straight or heterosexual,"Bachelor’s degree (BA, BS, B.Eng., etc.)",White or of European descent,35 - 44 years old,Yes,NaN,The survey was an appropriate length,Somewhat easy
1,Yes,No,South Africa,"Yes, part-time",Employed full-time,Some college/university study without earning ...,"Computer science, computer engineering, or sof...","10,000 or more employees",Data or business analyst;Desktop or enterprise...,6-8 years,0-2 years,Slightly satisfied,Moderately satisfied,Working in a different or more specialized tec...,"I’m not actively looking, but I am open to new...",Between 1 and 2 years ago,8.0,5.0,7.0,1.0,2.0,6.0,4.0,3.0,10.0,9.0,1.0,10.0,2.0,4.0,8.0,3.0,11.0,7.0,5.0,9.0,6.0,2.0,1.0,4.0,5.0,3.0,7.0,3.0,6.0,2.0,1.0,4.0,5.0,My job status or other personal status changed,S

In [9]:
#カテゴリ列について、trainingにしか現れないカテゴリ、もしくはtestにしか現れないカテゴリを可視化
def check_cat(df_train, df_test, col):
  train_s = set(df_train[col].unique())
  test_s = set(df_test[col].unique())
  only_training = train_s - test_s
  only_test = test_s - train_s

  print("Trainingのカテゴリ:" + str(train_s))
  print("Testのカテゴリ:" + str(test_s))
  print("trainingのみ:"+ str(only_training))
  print("testのみ:"+str(only_test))
  return only_training, only_test


only_training, only_test = check_cat(df_train, df_test, "Country")

Trainingのカテゴリ:{'Iceland', 'Indonesia', 'Slovenia', 'Barbados', 'Brazil', 'Japan', 'Bangladesh', 'Peru', 'Iraq', 'South Africa', 'Kuwait', 'Fiji', 'Viet Nam', 'Tunisia', 'Ecuador', 'Republic of Korea', 'Eritrea', 'Italy', 'Argentina', 'Spain', 'Marshall Islands', 'Egypt', 'Cameroon', 'Malawi', 'Finland', 'Uruguay', 'Afghanistan', 'Algeria', 'Oman', 'Cuba', 'Swaziland', 'Israel', 'Syrian Arab Republic', 'Albania', 'China', 'Bulgaria', 'Guyana', 'Norway', 'Switzerland', 'Kazakhstan', 'Iran, Islamic Republic of...', 'Nepal', 'Botswana', 'Gambia', 'Ghana', 'Saudi Arabia', 'Bosnia and Herzegovina', 'Senegal', 'Kyrgyzstan', 'Yemen', 'Estonia', 'Nicaragua', 'Bahamas', 'Sri Lanka', 'Hong Kong (S.A.R.)', 'Taiwan', 'Georgia', 'Nigeria', 'Dominican Republic', 'Romania', 'Sierra Leone', 'Mozambique', 'Bahrain', 'Mauritius', 'Paraguay', 'Congo, Republic of the...', 'United States', 'Russian Federation', "Côte d'Ivoire", 'Czech Republic', 'Montenegro', 'Belgium', 'Thailand', 'Uzbekistan', 'Honduras',

In [10]:
# カテゴリの値の出現数を見る
col = "YearsCoding"
df_train[col].value_counts()

6-8 years           9142
3-5 years           8973
9-11 years          6219
12-14 years         4413
15-17 years         3355
0-2 years           3022
18-20 years         2838
30 or more years    1850
21-23 years         1478
24-26 years         1030
27-29 years          555
Name: YearsCoding, dtype: int64

In [11]:
df_test[col].value_counts()

6-8 years           1092
3-5 years            815
9-11 years           805
12-14 years          585
15-17 years          473
18-20 years          330
30 or more years     179
0-2 years            166
21-23 years          158
24-26 years          129
27-29 years           76
Name: YearsCoding, dtype: int64

In [12]:
#カテゴリ特徴量について、トレーニングに一定数以上いない物をまとめてOthersに落とす。
thresh = 10

def cat_others(df_train, df_test, thresh, col):
  n_train = df_train[col].value_counts()
  n_train = n_train.reset_index()
  lis = list(n_train[n_train[col] <  thresh ]["index"])
  for x in lis:
    df_train[col].replace({x : "Others"}, inplace = True)
    df_test[col].replace({x : "Others"}, inplace = True)
  
  return df_train, df_test

df_train, df_test = cat_others( df_train, df_test, thresh, col )
df_train.head()


,Hobby,OpenSource,Country,Student,Employment,FormalEducation,UndergradMajor,CompanySize,DevType,YearsCoding,YearsCodingProf,JobSatisfaction,CareerSatisfaction,HopeFiveYears,JobSearchStatus,LastNewJob,AssessJob1,AssessJob2,AssessJob3,AssessJob4,AssessJob5,AssessJob6,AssessJob7,AssessJob8,AssessJob9,AssessJob10,AssessBenefits1,AssessBenefits2,AssessBenefits3,AssessBenefits4,AssessBenefits5,AssessBenefits6,AssessBenefits7,AssessBenefits8,AssessBenefits9,AssessBenefits10,AssessBenefits11,JobContactPriorities1,JobContactPriorities2,JobContactPriorities3,JobContactPriorities4,JobContactPriorities5,JobEmailPriorities1,JobEmailPriorities2,JobEmailPriorities3,JobEmailPriorities4,JobEmailPriorities5,JobEmailPriorities6,JobEmailPriorities7,UpdateCV,Currency,SalaryType,CurrencySymbol,CommunicationTools,TimeFullyProductive,TimeAfterBootcamp,AgreeDisagree1,AgreeDisagree2,AgreeDisagree3,FrameworkWorkedWith,OperatingSystem,NumberMonitors,CheckInCode,AdBlocker,AdBlockerDisable,AdsAgreeDisagree1,AdsAgreeDisagree2,AdsAgreeDisagree3,AdsActions,AdsPriorities1,AdsPriorities2,AdsPriorities3,AdsPriorities4,AdsPriorities5,AdsPriorities6,AdsPriorities7,AIDangerous,AIInteresting,AIResponsible,AIFuture,EthicsChoice,EthicsReport,EthicsResponsible,EthicalImplications,StackOverflowRecommend,StackOverflowVisit,StackOverflowHasAccount,StackOverflowParticipate,StackOverflowJobs,StackOverflowDevStory,StackOverflowJobsRecommend,StackOverflowConsiderMember,HypotheticalTools1,HypotheticalTools2,HypotheticalTools3,HypotheticalTools4,HypotheticalTools5,WakeTime,HoursComputer,HoursOutside,SkipMeals,ErgonomicDevices,Exercise,Gender,SexualOrientation,EducationParents,RaceEthnicity,Age,Dependents,MilitaryUS,SurveyTooLong,SurveyEasy
0,Yes,Yes,United Kingdom,No,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","A natural science (ex. biology, chemistry, phy...","10,000 or more employees",Database administrator;DevOps specialist;Full-...,30 or more years,18-20 years,Moderately dissatisfied,Neither satisfied nor dissatisfied,Working in a different or more specialized tec...,I am actively looking for a job,More than 4 years ago,1.0,7.0,10.0,8.0,2.0,5.0,4.0,3.0,6.0,9.0,1.0,5.0,3.0,7.0,10.0,4.0,11.0,9.0,6.0,2.0,8.0,3.0,1.0,5.0,2.0,4.0,1.0,3.0,4.0,5.0,2.0,6.0,7.0,I saw an employer’s advertisement,British pounds sterling (£),Yearly,GBP,Confluence;Office / productivity suite (Micros...,One to three months,NaN,Agree,Agree,Neither Agree nor Disagree,Django,Linux-based,2,A few times per week,Yes,Yes,Somewhat agree,Neither agree nor disagree,Neither agree nor disagree,NaN,3.0,5.0,1.0,4.0,6.0,7.0,2.0,Increasing automation of jobs,Increasing automation of jobs,The developers or the people creating the AI,I'm excited about the possibilities more than ...,Depends on what it is,Depends on what it is,Upper management at the company/organization,Yes,10 (Very Likely),A few times per month or weekly,Yes,A few times per month or weekly,Yes,"No, I have one but it's out of date",7,Yes,A little bit interested,A little bit interested,A little bit interested,A little bit interested,A little bit interested,Between 6:01 - 7:00 AM,5 - 8 hours,30 - 59 minutes,Never,Ergonomic keyboard or mouse,Daily or almost every day,Male,Straight or heterosexual,"Bachelor’s degree (BA, BS, B.Eng., etc.)",White or of European descent,35 - 44 years old,Yes,NaN,The survey was an appropriate length,Somewhat easy
1,Yes,No,South Africa,"Yes, part-time",Employed full-time,Some college/university study without earning ...,"Computer science, computer engineering, or sof...","10,000 or more employees",Data or business analyst;Desktop or enterprise...,6-8 years,0-2 years,Slightly satisfied,Moderately satisfied,Working in a different or more specialized tec...,"I’m not actively looking, but I am open to new...",Between 1 and 2 years ago,8.0,5.0,7.0,1.0,2.0,6.0,4.0,3.0,10.0,9.0,1.0,10.0,2.0,4.0,8.0,3.0,11.0,7.0,5.0,9.0,6.0,2.0,1.0,4.0,5.0,3.0,7.0,3.0,6.0,2.0,1.0,4.0,5.0,My job status or other personal status changed,S

In [13]:
# トレーニングデータにしか無い、テストデータにしか無いカテゴリをOthersに落とす。 #利用は要注意。本当は何か汎用的な特徴量に分解することなどを考える。未確認で実施してうっかり全部カテゴリを落とすとか笑える。
def cat_unseen(df_train, df_test, col):
  train_s = set(df_train[col].unique())
  test_s = set(df_test[col].unique())
  only_training = train_s - test_s
  only_test = test_s - train_s
  drop_others = only_training | only_test
  print(drop_others)
  for x in list(drop_others):
    df_train[col].replace({x : "Others"}, inplace = True)
    df_test[col].replace({x : "Others"}, inplace = True)
  
  return df_train, df_test

col = "YearsCoding"
df_train, df_test = cat_unseen( df_train, df_test, col )

set()


In [14]:
#次のセルの準備。本来数字のカテゴリ特徴量を数字に変換する準備。個別に確認して検討していく。
t_col = "CompanySize"
check_cat(df_train, df_test, t_col)

Trainingのカテゴリ:{nan, '500 to 999 employees', '5,000 to 9,999 employees', '10,000 or more employees', 'Fewer than 10 employees', '1,000 to 4,999 employees', '100 to 499 employees', '20 to 99 employees', '10 to 19 employees'}
Testのカテゴリ:{nan, '500 to 999 employees', '5,000 to 9,999 employees', '10,000 or more employees', 'Fewer than 10 employees', '100 to 499 employees', '1,000 to 4,999 employees', '20 to 99 employees', '10 to 19 employees'}
trainingのみ:set()
testのみ:set()


(set(), set())

In [15]:
#本来数字であるカテゴリ特徴量を数字に変換する。lis_cols_catから抜き、lis_cols_numにいれる
def order_num(df, col, lis):
  now_lis = df[col].unique()
  dic = {}
  for a, b in zip(now_lis, lis):
    dic[a] = b
  df[col].replace(dic, inplace = True)
  return df

t_col = "CompanySize"
order_lis=[10000, 10, 100, 20, 1000, 500, 5000, 0, np.nan]
df_train = order_num(df_train, t_col, order_lis)
df_test = order_num(df_test, t_col, order_lis)
lis_cols_cat.remove(t_col)
lis_cols_num.append(t_col)
df_train[t_col].unique()

array([10000.,    10.,   100.,    20.,  1000.,   500.,  5000.,     0.,
          nan])

In [16]:
# カテゴリリストの中からテキストか判別する。trainingまたはtestにthreshold以上のカーディナリティをテキストにする。でもtrainingとtestで数が違うならむしろデータをチェックした方が良い。
threshold = 100

def find_text(df, df2, col_lis, thre):
  txt_lis = []
  new_col_lis=[]
  for i, col in enumerate(col_lis):
    n1 = df[col].nunique()
    n2 = df2[col].nunique()
    print(i, col, "// training:", n1, "test:", n2)
    if n1 >= thre or n2> thre:
      txt_lis.append(col)
    else:
      new_col_lis.append(col)
  return new_col_lis, txt_lis

lis_cols_cat, lis_cols_txt = find_text(df_train, df_test, lis_cols_cat, threshold)
print(lis_cols_cat, lis_cols_txt)

0 Hobby // training: 2 test: 2
1 OpenSource // training: 2 test: 2
2 Country // training: 155 test: 2
3 Student // training: 3 test: 3
4 Employment // training: 6 test: 6
5 FormalEducation // training: 9 test: 9
6 UndergradMajor // training: 12 test: 12
7 DevType // training: 5695 test: 1368
8 YearsCoding // training: 11 test: 11
9 YearsCodingProf // training: 11 test: 11
10 JobSatisfaction // training: 7 test: 7
11 CareerSatisfaction // training: 7 test: 7
12 HopeFiveYears // training: 7 test: 7
13 JobSearchStatus // training: 3 test: 3
14 LastNewJob // training: 5 test: 5
15 UpdateCV // training: 8 test: 8
16 Currency // training: 19 test: 10
17 SalaryType // training: 3 test: 3
18 CurrencySymbol // training: 111 test: 11
19 CommunicationTools // training: 995 test: 422
20 TimeFullyProductive // training: 6 test: 6
21 TimeAfterBootcamp // training: 8 test: 8
22 AgreeDisagree1 // training: 5 test: 5
23 AgreeDisagree2 // training: 5 test: 5
24 AgreeDisagree3 // training: 5 test: 5
25 F

In [17]:
lis_cols_txt

['Country',
 'DevType',
 'CurrencySymbol',
 'CommunicationTools',
 'FrameworkWorkedWith']

In [18]:
# テキストの特徴量に対してElastic-netでstacked predictionでメタ特徴量を作成する。
# 分類にするならLogisticRegressionのペナルティを編集してElastic-netに。https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression
# Foldを変えるならはここを参照 https://scikit-learn.org/stable/modules/classes.html#module-sklearn.model_selection
# 線形モデルを変えるならここを参照 https://scikit-learn.org/stable/modules/classes.html#module-sklearn.linear_model
# メトリクスを変えるならここを参照https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from scipy.sparse import hstack
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error
# from sklearn.metrics import roc_auc_score

SEED = 71
NFOLDS = 5
fold = StratifiedKFold(n_splits=NFOLDS, random_state=SEED, shuffle=True)

def text_convert(df_train, df_train_y, df_test, col, fold):
  txt_train = df_train[col]
  txt_train.fillna("nan", inplace = True)
  txt_test = df_test[col]
  txt_test.fillna("nan", inplace = True)
  
  tfidf = TfidfVectorizer(max_features=10000)
  txt_train = tfidf.fit_transform(txt_train)
  txt_test = tfidf.transform(txt_test)

  oof_train = np.zeros(len(df_train))
  oof_test = np.zeros(len(df_test))

  for i, (train_idx, test_idx) in enumerate(fold.split(df_train, df_train_y)):
      train_x, train_y = txt_train[train_idx], df_train_y.iloc[train_idx]
      test_x, test_y = txt_train[test_idx], df_train_y.iloc[test_idx]
      
      # # LogisticRegression for 分類問題
      # clf = LogisticRegression(penalty='elasticnet',solver='sag')
      # clf.fit(X_tr, y_tr)
      # y_pred = clf.predict_proba(test_x)[:,1]
      # score = roc_auc_score(test_y, y_pred)
      # print('CV Score of Fold_%d is %f' % (i, score))
      # oof_test += clf.predict_proba(txt_test)[:,1]

      # ElasticNet for 回帰問題
      clf = ElasticNet(random_state=0)
      clf.fit(train_x, train_y)
      y_pred = clf.predict(test_x)
      # score = mean_squared_error(test_y, y_pred)
      # print('CV Score of Fold_%d is %f' % (i, score))
      oof_test += clf.predict(txt_test)

      oof_train[test_idx] = y_pred
  oof_test /= NFOLDS

  df_train[col+"_pred"] = oof_train
  df_test[col+"_pred"] = oof_test
  df_train.drop(columns = col, inplace=True)
  df_test.drop(columns = col, inplace=True)
  return df_train, df_test

for col in lis_cols_txt:
  df_train, df_test = text_convert(df_train, df_train_y, df_test, col, fold)
df_train.head()

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in 

,Hobby,OpenSource,Student,Employment,FormalEducation,UndergradMajor,CompanySize,YearsCoding,YearsCodingProf,JobSatisfaction,CareerSatisfaction,HopeFiveYears,JobSearchStatus,LastNewJob,AssessJob1,AssessJob2,AssessJob3,AssessJob4,AssessJob5,AssessJob6,AssessJob7,AssessJob8,AssessJob9,AssessJob10,AssessBenefits1,AssessBenefits2,AssessBenefits3,AssessBenefits4,AssessBenefits5,AssessBenefits6,AssessBenefits7,AssessBenefits8,AssessBenefits9,AssessBenefits10,AssessBenefits11,JobContactPriorities1,JobContactPriorities2,JobContactPriorities3,JobContactPriorities4,JobContactPriorities5,JobEmailPriorities1,JobEmailPriorities2,JobEmailPriorities3,JobEmailPriorities4,JobEmailPriorities5,JobEmailPriorities6,JobEmailPriorities7,UpdateCV,Currency,SalaryType,TimeFullyProductive,TimeAfterBootcamp,AgreeDisagree1,AgreeDisagree2,AgreeDisagree3,OperatingSystem,NumberMonitors,CheckInCode,AdBlocker,AdBlockerDisable,AdsAgreeDisagree1,AdsAgreeDisagree2,AdsAgreeDisagree3,AdsActions,AdsPriorities1,AdsPriorities2,AdsPriorities3,AdsPriorities4,AdsPriorities5,AdsPriorities6,AdsPriorities7,AIDangerous,AIInteresting,AIResponsible,AIFuture,EthicsChoice,EthicsReport,EthicsResponsible,EthicalImplications,StackOverflowRecommend,StackOverflowVisit,StackOverflowHasAccount,StackOverflowParticipate,StackOverflowJobs,StackOverflowDevStory,StackOverflowJobsRecommend,StackOverflowConsiderMember,HypotheticalTools1,HypotheticalTools2,HypotheticalTools3,HypotheticalTools4,HypotheticalTools5,WakeTime,HoursComputer,HoursOutside,SkipMeals,ErgonomicDevices,Exercise,Gender,SexualOrientation,EducationParents,RaceEthnicity,Age,Dependents,MilitaryUS,SurveyTooLong,SurveyEasy,Country_pred,DevType_pred,CurrencySymbol_pred,CommunicationTools_pred,FrameworkWorkedWith_pred
0,Yes,Yes,No,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","A natural science (ex. biology, chemistry, phy...",10000.0,30 or more years,18-20 years,Moderately dissatisfied,Neither satisfied nor dissatisfied,Working in a different or more specialized tec...,I am actively looking for a job,More than 4 years ago,1.0,7.0,10.0,8.0,2.0,5.0,4.0,3.0,6.0,9.0,1.0,5.0,3.0,7.0,10.0,4.0,11.0,9.0,6.0,2.0,8.0,3.0,1.0,5.0,2.0,4.0,1.0,3.0,4.0,5.0,2.0,6.0,7.0,I saw an employer’s advertisement,British pounds sterling (£),Yearly,One to three months,NaN,Agree,Agree,Neither Agree nor Disagree,Linux-based,2,A few times per week,Yes,Yes,Somewhat agree,Neither agree nor disagree,Neither agree nor disagree,NaN,3.0,5.0,1.0,4.0,6.0,7.0,2.0,Increasing automation of jobs,Increasing automation of jobs,The developers or the people creating the AI,I'm excited about the possibilities more than ...,Depends on what it is,Depends on what it is,Upper management at the company/organization,Yes,10 (Very Likely),A few times per month or weekly,Yes,A few times per month or weekly,Yes,"No, I have one but it's out of date",7,Yes,A little bit interested,A little bit interested,A little bit interested,A little bit interested,A little bit interested,Between 6:01 - 7:00 AM,5 - 8 hours,30 - 59 minutes,Never,Ergonomic keyboard or mouse,Daily or almost every day,Male,Straight or heterosexual,"Bachelor’s degree (BA, BS, B.Eng., etc.)",White or of European descent,35 - 44 years old,Yes,NaN,The survey was an appropriate length,Somewhat easy,101258.352106,97988.065984,95651.377054,99733.989868,95904.072252
1,Yes,No,"Yes, part-time",Employed full-time,Some college/university study without earning ...,"Computer science, computer engineering, or sof...",10000.0,6-8 years,0-2 years,Slightly satisfied,Moderately satisfied,Working in a different or more specialized tec...,"I’m not actively looking, but I am open to new...",Between 1 and 2 years ago,8.0,5.0,7.0,1.0,2.0,6.0,4.0,3.0,10.0,9.0,1.0,10.0,2.0,4.0,8.0,3.0,11.0,7.0,5.0,9.0,6.0,2.0,1.0,4.0,5.0,3.0,7.0,3.0,6.0,2.0,1.0,4.0,5.0,My job status or other personal status changed,South African rands (R),Yearly,Three to six months,NaN,Strongly agree,Agree,Strongly disagree,Windows,2,Weekly or a few times per month,N

In [19]:
# カテゴリ特徴量の変換　https://www.analyticsvidhya.com/blog/2020/08/types-of-categorical-data-encoding/
# https://contrib.scikit-learn.org/category_encoders/
import category_encoders as ce
import warnings
warnings.simplefilter('ignore')

def cat_encode_for_liner(df, df2, col):
  encode = ce.OneHotEncoder(cols = col, handle_unknown = 'error', use_cat_names = True) #初回は初見のカテゴリを調べるために"error"で。
  df_tmp = encode.fit_transform(df[col])
  df = pd.concat([df, df_tmp], axis = 1)
  df.drop(columns = col, inplace = True)
  df_tmp = encode.transform(df2[col])
  df2 = pd.concat([df2, df_tmp], axis = 1)
  df2.drop(columns = col, inplace = True)
  return df, df2

for col in lis_cols_cat:
  df_train, df_test = cat_encode_for_liner(df_train, df_test, col)
print(df_train.shape, df_test.shape)
df_train.head()

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


(42893, 567) (4809, 567)


,CompanySize,AssessJob1,AssessJob2,AssessJob3,AssessJob4,AssessJob5,AssessJob6,AssessJob7,AssessJob8,AssessJob9,AssessJob10,AssessBenefits1,AssessBenefits2,AssessBenefits3,AssessBenefits4,AssessBenefits5,AssessBenefits6,AssessBenefits7,AssessBenefits8,AssessBenefits9,AssessBenefits10,AssessBenefits11,JobContactPriorities1,JobContactPriorities2,JobContactPriorities3,JobContactPriorities4,JobContactPriorities5,JobEmailPriorities1,JobEmailPriorities2,JobEmailPriorities3,JobEmailPriorities4,JobEmailPriorities5,JobEmailPriorities6,JobEmailPriorities7,AdsPriorities1,AdsPriorities2,AdsPriorities3,AdsPriorities4,AdsPriorities5,AdsPriorities6,AdsPriorities7,Country_pred,DevType_pred,CurrencySymbol_pred,CommunicationTools_pred,FrameworkWorkedWith_pred,Hobby_Yes,Hobby_No,OpenSource_Yes,OpenSource_No,Student_No,"Student_Yes, part-time","Student_Yes, full-time",Student_nan,Employment_Employed full-time,Employment_Employed part-time,"Employment_Independent contractor, freelancer, or self-employed","Employment_Not employed, and not looking for work",Employment_nan,"Employment_Not employed, but looking for work",Employment_Retired,"FormalEducation_Bachelor’s degree (BA, BS, B.Eng., etc.)",FormalEducation_Some college/university study without earning a degree,"FormalEducation_Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)","FormalEducation_Master’s degree (MA, MS, M.Eng., MBA, etc.)",FormalEducation_Associate degree,"FormalEducation_Other doctoral degree (Ph.D, Ed.D., etc.)",FormalEducation_nan,"FormalEducation_Professional degree (JD, MD, etc.)",FormalEducation_Primary/elementary school,FormalEducation_I never completed any formal education,"UndergradMajor_A natural science (ex. biology, chemistry, physics)","UndergradMajor_Computer science, computer engineering, or software engineering","UndergradMajor_Fine arts or performing arts (ex. graphic design, music, studio art)",UndergradMajor_nan,"UndergradMajor_A business discipline (ex. accounting, finance, marketing)","UndergradMajor_Another engineering discipline (ex. civil, electrical, mechanical)","UndergradMajor_A social science (ex. anthropology, psychology, political science)",UndergradMajor_Web development or web design,"UndergradMajor_Information systems, information technology, or system administration","UndergradMajor_A humanities discipline (ex. literature, history, philosophy)",UndergradMajor_Mathematics or statistics,"UndergradMajor_A health science (ex. nursing, pharmacy, radiology)",UndergradMajor_I never declared a major,YearsCoding_30 or more years,YearsCoding_6-8 years,YearsCoding_9-11 years,YearsCoding_0-2 years,YearsCoding_15-17 years,YearsCoding_18-20 years,YearsCoding_3-5 years,YearsCoding_12-14 years,YearsCoding_24-26 years,YearsCoding_21-23 years,YearsCoding_27-29 years,YearsCoding_nan,YearsCodingProf_18-20 years,YearsCodingProf_0-2 years,YearsCodingProf_3-5 years,YearsCodingProf_21-23 years,YearsCodingProf_12-14 years,YearsCodingProf_6-8 years,YearsCodingProf_9-11 years,YearsCodingProf_24-26 years,YearsCodingProf_15-17 years,YearsCodingProf_30 or more years,YearsCodingProf_27-29 years,YearsCodingProf_nan,JobSatisfaction_Moderately dissatisfied,JobSatisfaction_Slightly satisfied,JobSatisfaction_Moderately satisfied,JobSatisfaction_Neither satisfied nor dissatisfied,JobSatisfaction_Slightly dissatisfied,JobSatisfaction_Extremely dissatisfied,JobSatisfaction_nan,JobSatisfaction_Extremely satisfied,CareerSatisfaction_Neither satisfied nor dissatisfied,CareerSatisfaction_Moderately satisfied,CareerSatisfaction_Slightly satisfied,CareerSatisfaction_Moderately dissatisfied,CareerSatisfaction_Slightly dissatisfied,CareerSatisfaction_Extremely satisfied,CareerSatisfaction_Extremely dissatisfied,CareerSatisfaction_nan,HopeFiveYears_Working in a different or more specialized technical role than the one I'm in now,HopeFiveYears_Working as a founder or co-founder of my own company,HopeFiveYears_Doing the same work,HopeFiveYears_Working in a career 

In [20]:
#欠損値補完の前に欠損値カウント
def null_count(df):
  df["null_count"] = df.isnull().sum(axis=1)
  return df

df_train = null_count(df_train)
df_test = null_count(df_test)
df_train.head()

,CompanySize,AssessJob1,AssessJob2,AssessJob3,AssessJob4,AssessJob5,AssessJob6,AssessJob7,AssessJob8,AssessJob9,AssessJob10,AssessBenefits1,AssessBenefits2,AssessBenefits3,AssessBenefits4,AssessBenefits5,AssessBenefits6,AssessBenefits7,AssessBenefits8,AssessBenefits9,AssessBenefits10,AssessBenefits11,JobContactPriorities1,JobContactPriorities2,JobContactPriorities3,JobContactPriorities4,JobContactPriorities5,JobEmailPriorities1,JobEmailPriorities2,JobEmailPriorities3,JobEmailPriorities4,JobEmailPriorities5,JobEmailPriorities6,JobEmailPriorities7,AdsPriorities1,AdsPriorities2,AdsPriorities3,AdsPriorities4,AdsPriorities5,AdsPriorities6,AdsPriorities7,Country_pred,DevType_pred,CurrencySymbol_pred,CommunicationTools_pred,FrameworkWorkedWith_pred,Hobby_Yes,Hobby_No,OpenSource_Yes,OpenSource_No,Student_No,"Student_Yes, part-time","Student_Yes, full-time",Student_nan,Employment_Employed full-time,Employment_Employed part-time,"Employment_Independent contractor, freelancer, or self-employed","Employment_Not employed, and not looking for work",Employment_nan,"Employment_Not employed, but looking for work",Employment_Retired,"FormalEducation_Bachelor’s degree (BA, BS, B.Eng., etc.)",FormalEducation_Some college/university study without earning a degree,"FormalEducation_Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)","FormalEducation_Master’s degree (MA, MS, M.Eng., MBA, etc.)",FormalEducation_Associate degree,"FormalEducation_Other doctoral degree (Ph.D, Ed.D., etc.)",FormalEducation_nan,"FormalEducation_Professional degree (JD, MD, etc.)",FormalEducation_Primary/elementary school,FormalEducation_I never completed any formal education,"UndergradMajor_A natural science (ex. biology, chemistry, physics)","UndergradMajor_Computer science, computer engineering, or software engineering","UndergradMajor_Fine arts or performing arts (ex. graphic design, music, studio art)",UndergradMajor_nan,"UndergradMajor_A business discipline (ex. accounting, finance, marketing)","UndergradMajor_Another engineering discipline (ex. civil, electrical, mechanical)","UndergradMajor_A social science (ex. anthropology, psychology, political science)",UndergradMajor_Web development or web design,"UndergradMajor_Information systems, information technology, or system administration","UndergradMajor_A humanities discipline (ex. literature, history, philosophy)",UndergradMajor_Mathematics or statistics,"UndergradMajor_A health science (ex. nursing, pharmacy, radiology)",UndergradMajor_I never declared a major,YearsCoding_30 or more years,YearsCoding_6-8 years,YearsCoding_9-11 years,YearsCoding_0-2 years,YearsCoding_15-17 years,YearsCoding_18-20 years,YearsCoding_3-5 years,YearsCoding_12-14 years,YearsCoding_24-26 years,YearsCoding_21-23 years,YearsCoding_27-29 years,YearsCoding_nan,YearsCodingProf_18-20 years,YearsCodingProf_0-2 years,YearsCodingProf_3-5 years,YearsCodingProf_21-23 years,YearsCodingProf_12-14 years,YearsCodingProf_6-8 years,YearsCodingProf_9-11 years,YearsCodingProf_24-26 years,YearsCodingProf_15-17 years,YearsCodingProf_30 or more years,YearsCodingProf_27-29 years,YearsCodingProf_nan,JobSatisfaction_Moderately dissatisfied,JobSatisfaction_Slightly satisfied,JobSatisfaction_Moderately satisfied,JobSatisfaction_Neither satisfied nor dissatisfied,JobSatisfaction_Slightly dissatisfied,JobSatisfaction_Extremely dissatisfied,JobSatisfaction_nan,JobSatisfaction_Extremely satisfied,CareerSatisfaction_Neither satisfied nor dissatisfied,CareerSatisfaction_Moderately satisfied,CareerSatisfaction_Slightly satisfied,CareerSatisfaction_Moderately dissatisfied,CareerSatisfaction_Slightly dissatisfied,CareerSatisfaction_Extremely satisfied,CareerSatisfaction_Extremely dissatisfied,CareerSatisfaction_nan,HopeFiveYears_Working in a different or more specialized technical role than the one I'm in now,HopeFiveYears_Working as a founder or co-founder of my own company,HopeFiveYears_Doing the same work,HopeFiveYears_Working in a career 

In [21]:
#数値型特徴量の欠損値補完を行う。欠損値がある列には欠損フラグの列も追加 #純粋な欠損値だけ見ている。データを確認し、偽装欠損値と見込まれる値が見つかれば別途対応
from sklearn.impute import SimpleImputer

def impute_and_flag(df_train, df_test, col_lis):
  imp_lis = []
  for col in col_lis:
    if df_train[col].isnull().any() == True:
      imp_lis.append(col)
      df_train[col+"_null"] = 0
      df_train.loc[df_train[col].isnull(), col+"_null"] = 1
      df_test[col+"_null"] = 0
      df_test.loc[df_test[col].isnull(), col+"_null"] = 1
      imp = SimpleImputer(missing_values = np.nan, strategy = 'median')
      df_train[col] = imp.fit_transform(df_train[[col]])
      df_test[col] = imp.transform(df_test[[col]])
    elif df_train[col].isnull().any() == True:
      print("missing values is found in only test data for : "+ col)
      #欠損値がTestデータにだけ現れるケースはデータを要確認。そもそもその列は使えないかも。
    else:
      pass
  print("Imputed :"+ str(imp_lis))
  return df_train, df_test


df_train, df_test = impute_and_flag(df_train, df_test, lis_cols_num)

Imputed :['AssessJob1', 'AssessJob2', 'AssessJob3', 'AssessJob4', 'AssessJob5', 'AssessJob6', 'AssessJob7', 'AssessJob8', 'AssessJob9', 'AssessJob10', 'AssessBenefits1', 'AssessBenefits2', 'AssessBenefits3', 'AssessBenefits4', 'AssessBenefits5', 'AssessBenefits6', 'AssessBenefits7', 'AssessBenefits8', 'AssessBenefits9', 'AssessBenefits10', 'AssessBenefits11', 'JobContactPriorities1', 'JobContactPriorities2', 'JobContactPriorities3', 'JobContactPriorities4', 'JobContactPriorities5', 'JobEmailPriorities1', 'JobEmailPriorities2', 'JobEmailPriorities3', 'JobEmailPriorities4', 'JobEmailPriorities5', 'JobEmailPriorities6', 'JobEmailPriorities7', 'AdsPriorities1', 'AdsPriorities2', 'AdsPriorities3', 'AdsPriorities4', 'AdsPriorities5', 'AdsPriorities6', 'AdsPriorities7', 'CompanySize']


In [22]:
#数値型特徴量の標準化　https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing
from sklearn.preprocessing import StandardScaler

def stand(df_train, df_test, lis_cols_num):
  for col in lis_cols_num:
    scaler = StandardScaler()
    df_train[col] = scaler.fit_transform(df_train[[col]])
    df_test[col] = scaler.transform(df_test[[col]])
  return df_train, df_test

df_train, df_test = stand(df_train, df_test, lis_cols_num)
df_train.head()

,CompanySize,AssessJob1,AssessJob2,AssessJob3,AssessJob4,AssessJob5,AssessJob6,AssessJob7,AssessJob8,AssessJob9,AssessJob10,AssessBenefits1,AssessBenefits2,AssessBenefits3,AssessBenefits4,AssessBenefits5,AssessBenefits6,AssessBenefits7,AssessBenefits8,AssessBenefits9,AssessBenefits10,AssessBenefits11,JobContactPriorities1,JobContactPriorities2,JobContactPriorities3,JobContactPriorities4,JobContactPriorities5,JobEmailPriorities1,JobEmailPriorities2,JobEmailPriorities3,JobEmailPriorities4,JobEmailPriorities5,JobEmailPriorities6,JobEmailPriorities7,AdsPriorities1,AdsPriorities2,AdsPriorities3,AdsPriorities4,AdsPriorities5,AdsPriorities6,AdsPriorities7,Country_pred,DevType_pred,CurrencySymbol_pred,CommunicationTools_pred,FrameworkWorkedWith_pred,Hobby_Yes,Hobby_No,OpenSource_Yes,OpenSource_No,Student_No,"Student_Yes, part-time","Student_Yes, full-time",Student_nan,Employment_Employed full-time,Employment_Employed part-time,"Employment_Independent contractor, freelancer, or self-employed","Employment_Not employed, and not looking for work",Employment_nan,"Employment_Not employed, but looking for work",Employment_Retired,"FormalEducation_Bachelor’s degree (BA, BS, B.Eng., etc.)",FormalEducation_Some college/university study without earning a degree,"FormalEducation_Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)","FormalEducation_Master’s degree (MA, MS, M.Eng., MBA, etc.)",FormalEducation_Associate degree,"FormalEducation_Other doctoral degree (Ph.D, Ed.D., etc.)",FormalEducation_nan,"FormalEducation_Professional degree (JD, MD, etc.)",FormalEducation_Primary/elementary school,FormalEducation_I never completed any formal education,"UndergradMajor_A natural science (ex. biology, chemistry, physics)","UndergradMajor_Computer science, computer engineering, or software engineering","UndergradMajor_Fine arts or performing arts (ex. graphic design, music, studio art)",UndergradMajor_nan,"UndergradMajor_A business discipline (ex. accounting, finance, marketing)","UndergradMajor_Another engineering discipline (ex. civil, electrical, mechanical)","UndergradMajor_A social science (ex. anthropology, psychology, political science)",UndergradMajor_Web development or web design,"UndergradMajor_Information systems, information technology, or system administration","UndergradMajor_A humanities discipline (ex. literature, history, philosophy)",UndergradMajor_Mathematics or statistics,"UndergradMajor_A health science (ex. nursing, pharmacy, radiology)",UndergradMajor_I never declared a major,YearsCoding_30 or more years,YearsCoding_6-8 years,YearsCoding_9-11 years,YearsCoding_0-2 years,YearsCoding_15-17 years,YearsCoding_18-20 years,YearsCoding_3-5 years,YearsCoding_12-14 years,YearsCoding_24-26 years,YearsCoding_21-23 years,YearsCoding_27-29 years,YearsCoding_nan,YearsCodingProf_18-20 years,YearsCodingProf_0-2 years,YearsCodingProf_3-5 years,YearsCodingProf_21-23 years,YearsCodingProf_12-14 years,YearsCodingProf_6-8 years,YearsCodingProf_9-11 years,YearsCodingProf_24-26 years,YearsCodingProf_15-17 years,YearsCodingProf_30 or more years,YearsCodingProf_27-29 years,YearsCodingProf_nan,JobSatisfaction_Moderately dissatisfied,JobSatisfaction_Slightly satisfied,JobSatisfaction_Moderately satisfied,JobSatisfaction_Neither satisfied nor dissatisfied,JobSatisfaction_Slightly dissatisfied,JobSatisfaction_Extremely dissatisfied,JobSatisfaction_nan,JobSatisfaction_Extremely satisfied,CareerSatisfaction_Neither satisfied nor dissatisfied,CareerSatisfaction_Moderately satisfied,CareerSatisfaction_Slightly satisfied,CareerSatisfaction_Moderately dissatisfied,CareerSatisfaction_Slightly dissatisfied,CareerSatisfaction_Extremely satisfied,CareerSatisfaction_Extremely dissatisfied,CareerSatisfaction_nan,HopeFiveYears_Working in a different or more specialized technical role than the one I'm in now,HopeFiveYears_Working as a founder or co-founder of my own company,HopeFiveYears_Doing the same work,HopeFiveYears_Working in a career 

In [23]:
# 連続値モデルのリスト。ハイパラは別途チューニング。
from sklearn.linear_model import ElasticNet
from sklearn.svm import LinearSVR

clf_lis = {}
# Elastic_net https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html 
clf_lis["ElasticNet"] = ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
           max_iter=1000, normalize=False, positive=False, precompute=False,
           random_state=0, selection='cyclic', tol=0.0001, warm_start=False)

# SVM https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVR.html#sklearn.svm.LinearSVR
clf_lis["LinearSVR"] = LinearSVR(C=1.0, dual=True, epsilon=0.0, fit_intercept=True,
          intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,
          random_state=0, tol=0.0001, verbose=0)

clf_lis

{'ElasticNet': ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
            max_iter=1000, normalize=False, positive=False, precompute=False,
            random_state=0, selection='cyclic', tol=0.0001, warm_start=False),
 'LinearSVR': LinearSVR(C=1.0, dual=True, epsilon=0.0, fit_intercept=True,
           intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,
           random_state=0, tol=0.0001, verbose=0)}

In [27]:
# 分類モデルのリスト
from sklearn.linear_model import LogisticRegression

clf_lis = {}

# ElasticNet相当 https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression
clf_lis["Elastic_net"] = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=0.5, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='elasticnet',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

clf_lis

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [25]:
#パーティショニングとStacked prediction
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
# Foldを変えるならはここを参照 https://scikit-learn.org/stable/modules/classes.html#module-sklearn.model_selection
# 線形モデルを変えるならここを参照 https://scikit-learn.org/stable/modules/classes.html#module-sklearn.linear_model
# メトリクスを変えるならここを参照https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics


SEED = 72
NFOLDS = 5
fold = StratifiedKFold(n_splits=NFOLDS, random_state=SEED, shuffle=True)


def CVprediction(df_train, df_train_y, df_test, fold, clf_lis):
  oof_train = np.zeros(len(df_train))
  oof_test = np.zeros(len(df_test))
  df_test_pred = pd.DataFrame()
  df_train_pred = pd.DataFrame()
  score_table = {}
      
  # # 分類問題(途中?)
  # for key, clf in clf_lis.items():
  #   for i, (train_idx, test_idx) in enumerate(fold.split(df_train, df_train_y)):
  #     train_x, train_y = df_train.iloc[train_idx], df_train_y.iloc[train_idx]
  #     test_x, test_y = df_train.iloc[test_idx], df_train_y.iloc[test_idx]
  #     clf.fit(train_x, train_y)
  #     y_pred = clf.predict_proba(test_x)[:,1]
  #     score = roc_auc_score(test_y, y_pred)
  #     print(key + ' : CV Score of Fold_%d is %f' % (i, score))
  #     oof_test += clf.predict_proba(txt_test)[:,1]
  #     oof_train[test_idx] = y_pred
  #   oof_test /= NFOLDS
  #   df_test_pred[key] = oof_test
  #   df_train_pred[key] = oof_train

  # 連続値問題
  for key, clf in clf_lis.items():
    for i, (train_idx, test_idx) in enumerate(fold.split(df_train, df_train_y)):
      train_x, train_y = df_train.iloc[train_idx], df_train_y.iloc[train_idx]
      test_x, test_y = df_train.iloc[test_idx], df_train_y.iloc[test_idx]
        
      clf.fit(train_x, train_y)
      y_pred = clf.predict(test_x)
      score = mean_squared_error(test_y, y_pred)
      print(key + ' : CV Score of Fold_%d is %f' % (i, score))
      score_table[key, i] = score
      oof_test += clf.predict(df_test)
      oof_train[test_idx] = y_pred
    oof_test /= NFOLDS
    df_test_pred[key] = oof_test
    df_train_pred[key] = oof_train
  
  # スコアのテーブルを整備
  df = pd.DataFrame()
  for k, v in score_table.items():
    df.loc[k[1], k[0]] = v
  df2 = df.agg(['sum', 'mean', 'min', 'max'])
  df_score = pd.concat([df, df2])

  return df_train_pred, df_test_pred, df_score


df_train_pred, df_test_pred, score_table = CVprediction(df_train, df_train_y, df_test, fold, clf_lis)
score_table

ElasticNet : CV Score of Fold_0 is 38633012033.786659
ElasticNet : CV Score of Fold_1 is 38229843938.037125
ElasticNet : CV Score of Fold_2 is 39512667612.906906
ElasticNet : CV Score of Fold_3 is 38461919260.539009
ElasticNet : CV Score of Fold_4 is 38346783856.076927
LinearSVR : CV Score of Fold_0 is 45875416201.051170
LinearSVR : CV Score of Fold_1 is 43580450436.938362
LinearSVR : CV Score of Fold_2 is 47728120993.841110
LinearSVR : CV Score of Fold_3 is 41312671677.183090
LinearSVR : CV Score of Fold_4 is 41443726177.183220


,ElasticNet,LinearSVR
0,3.863301e+10,4.587542e+10
1,3.822984e+10,4.358045e+10
2,3.951267e+10,4.772812e+10
3,3.846192e+10,4.131267e+10
4,3.834678e+10,4.144373e+10
sum,1.931842e+11,2.199404e+11
mean,3.863685e+10,4.398808e+10
min,3.822984e+10,4.131267e+10
max,3.951267e+10,4.772812e+10


In [ ]:
# ハイパラチューニング・・は別のノートで。
